In [1]:
import os
import shutil
import glob
import datetime
import time
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

In [2]:
# Read raw PPG files
mdf = pd.concat(map(pd.read_csv, glob.glob('../prof_data/resteaze/*.csv')))

In [3]:
mdf.reset_index(drop=True)
mdf

,Unnamed: 0,counter,unixTimes,accelerometerX,accelerometerY,accelerometerZ,capacitorsA,capacitorsB,capacitorsC,gyroscopeX,...,sleep_state,Leg Movement,Hypopnea,Arousal,RERA,Relative Desaturation,Obstructive Apnea,Central Apnea,Leg Movement (L),Leg Movement (R)
0,0,3,1644991436679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
1,1,0,1644991436719,74.0,335.0,3989.0,31863.0,NaN,NaN,-162.0,...,-1.0,0,0,0,0,0,0,0,0,0
2,2,4,1644991436719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
3,3,1,1644991436759,87.0,327.0,3988.0,31861.0,NaN,NaN,-171.0,...,-1.0,0,0,0,0,0,0,0,0,0
4,4,5,1644991436759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312000,1312000,1318599,1648123856516,-2191.0,164.0,3393.0,33246.0,NaN,NaN,24.0,...,2.0,0,0,0,0,0,0,0,0,0
1312001,1312001,1318600,1648123856555,-2195.0,163.0,3401.0,33243.0,NaN,NaN,6.0,...,2.0,0,0,0,0,0,0,0,0,0
1312002,1312002,1318601,1648123856588,-2197.0,163.0,3401.0,33245.0,NaN,NaN,25.0,...,2.0,0,0,0,0,0,0,0,0,0
1312003,1312003,1318602,1648123856628,-2190.0,173.0,3405.0,33247.0,NaN,NaN,19.0,...,2.0,0,0,0,0,0,0,0,0,0


In [4]:
all_df = mdf[[
    'unixTimes', 'accelerometerX', 'accelerometerY', 'accelerometerZ',
    'gyroscopeX', 'gyroscopeY', 'gyroscopeZ', 'ledGreen', 'sleep_state'
]]

all_df = all_df[all_df['sleep_state'].notna()]
df = all_df.interpolate()

In [5]:
df

,unixTimes,accelerometerX,accelerometerY,accelerometerZ,gyroscopeX,gyroscopeY,gyroscopeZ,ledGreen,sleep_state
0,1644991436679,NaN,NaN,NaN,NaN,NaN,NaN,2716.0,-1.0
1,1644991436719,74.0,335.0,3989.0,-162.0,-93.0,109.0,2711.5,-1.0
2,1644991436719,80.5,331.0,3988.5,-166.5,-65.0,123.0,2707.0,-1.0
3,1644991436759,87.0,327.0,3988.0,-171.0,-37.0,137.0,2704.5,-1.0
4,1644991436759,79.5,343.0,4003.5,-177.5,-24.0,134.0,2702.0,-1.0
...,...,...,...,...,...,...,...,...,...
1312000,1648123856516,-2191.0,164.0,3393.0,24.0,-233.0,58.0,7506.0,2.0
1312001,1648123856555,-2195.0,163.0,3401.0,6.0,-227.0,61.0,7506.0,2.0
1312002,1648123856588,-2197.0,163.0,3401.0,25.0,-223.0,64.0,7506.0,2.0
1312003,1648123856628,-2190.0,173.0,3405.0,19.0,-229.0,60.0,7506.0,2.0


In [6]:
# mdf = mdf.apply(lambda x: pd.Series(x.dropna().values))

df = df[df.sleep_state != -1].reset_index()


In [7]:
df = ((df + df.shift(-1)) / 2)[::2]

# Binary Classification
df["sleep_state"] = np.where(df["sleep_state"] == 0, 0, 1)

# fill NaN values
df = df.dropna()

df

,index,unixTimes,accelerometerX,accelerometerY,accelerometerZ,gyroscopeX,gyroscopeY,gyroscopeZ,ledGreen,sleep_state
0,12734.5,1.644992e+12,57.00,296.25,4021.00,-201.75,-47.75,99.00,2558.75,0
2,12736.5,1.644992e+12,29.00,302.50,4026.00,-226.25,-75.25,119.75,2548.75,0
4,12738.5,1.644992e+12,40.50,310.25,4031.00,-228.00,-108.25,113.25,2546.00,0
6,12740.5,1.644992e+12,53.50,310.00,4028.00,-224.50,-86.00,101.25,2545.25,0
8,12742.5,1.644992e+12,45.25,307.25,4026.75,-232.00,-86.25,116.50,2545.00,0
...,...,...,...,...,...,...,...,...,...,...
18364782,1311994.5,1.648124e+12,-2191.00,161.00,3397.00,12.50,-226.00,42.50,7492.00,1
18364784,1311996.5,1.648124e+12,-2189.50,157.50,3394.50,20.00,-222.50,61.00,7497.00,1
18364786,1311998.5,1.648124e+12,-2191.50,162.00,3393.00,20.00,-227.00,65.00,7502.50,1
18364788,1312000.5,1.648124e+12,-2193.00,163.50,3397.00,15.00,-230.00,59.50,7506.00,1


In [8]:
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from numpy import mean, sqrt, square, arange

# # RMS of accelerometer
# df['accelerometer'] = df[[
#     'accelerometerX', 'accelerometerY', 'accelerometerZ'
# ]].apply(lambda x: sqrt(square(x['accelerometerX']) + square(x['accelerometerY']) + square(x['accelerometerZ'])), axis=1)

# # RMS of gyroscope
# df['gyroscope'] = df[[
#     'gyroscopeX', 'gyroscopeY', 'gyroscopeZ'
# ]].apply(lambda x: sqrt(square(x['gyroscopeX']) + square(x['gyroscopeY']) + square(x['gyroscopeZ'])), axis=1)

# df = df.drop(columns=['accelerometerX', 'accelerometerY', 'accelerometerZ', 'gyroscopeX', 'gyroscopeY', 'gyroscopeZ'])

In [9]:
#@title Bandpass Filter
from scipy.signal import butter, sosfilt, sosfreqz


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y


# apply bandpass filter

fs = 25
lowcut = 0.55
highcut = 3.0

df['ledGreen'] = butter_bandpass_filter(df['ledGreen'],
                                            lowcut,
                                            highcut,
                                            fs,
                                            order=3)


In [10]:
# data = df[['ledGreen','accelerometer','gyroscope','sleep_state']]
data = df

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9182396 entries, 0 to 18364790
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   index           float64
 1   unixTimes       float64
 2   accelerometerX  float64
 3   accelerometerY  float64
 4   accelerometerZ  float64
 5   gyroscopeX      float64
 6   gyroscopeY      float64
 7   gyroscopeZ      float64
 8   ledGreen        float64
 9   sleep_state     int32  
dtypes: float64(9), int32(1)
memory usage: 735.6 MB


In [12]:
# data.to_csv('prefinal_processed.csv')

## Process Segmentwise HR data

In [13]:
# data = pd.read_csv("prefinal_processed.csv", index_col=0)

In [14]:
data

,index,unixTimes,accelerometerX,accelerometerY,accelerometerZ,gyroscopeX,gyroscopeY,gyroscopeZ,ledGreen,sleep_state
0,12734.5,1.644992e+12,57.00,296.25,4021.00,-201.75,-47.75,99.00,43.987752,0
2,12736.5,1.644992e+12,29.00,302.50,4026.00,-226.25,-75.25,119.75,243.038014,0
4,12738.5,1.644992e+12,40.50,310.25,4031.00,-228.00,-108.25,113.25,627.523616,0
6,12740.5,1.644992e+12,53.50,310.00,4028.00,-224.50,-86.00,101.25,1035.918959,0
8,12742.5,1.644992e+12,45.25,307.25,4026.75,-232.00,-86.25,116.50,1249.684019,0
...,...,...,...,...,...,...,...,...,...,...
18364782,1311994.5,1.648124e+12,-2191.00,161.00,3397.00,12.50,-226.00,42.50,18.356277,1
18364784,1311996.5,1.648124e+12,-2189.50,157.50,3394.50,20.00,-222.50,61.00,19.802624,1
18364786,1311998.5,1.648124e+12,-2191.50,162.00,3393.00,20.00,-227.00,65.00,20.153999,1
18364788,1312000.5,1.648124e+12,-2193.00,163.50,3397.00,15.00,-230.00,59.50,20.191107,1


In [15]:
import heartpy as hp

fs = 25  # Hz
epoch_length = 60  # seconds

# Calculate the breathing rate for each epoch
wd_seg, m_seg = hp.process_segmentwise(np.array(data['ledGreen']),
                                       sample_rate=fs,
                                       segment_width=epoch_length)

c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\numpy\ma\core.py:5244: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\lucki\anaconda3\envs\tf\lib\site

In [16]:
m_seg.keys()

dict_keys(['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'segment_indices'])

In [17]:
wd_seg.keys()

dict_keys(['hr', 'sample_rate', 'peaklist', 'ybeat', 'rolling_mean', 'RR_list', 'RR_indices', 'RR_diff', 'RR_sqdiff', 'rrsd', 'best', 'removed_beats', 'removed_beats_y', 'binary_peaklist', 'RR_masklist', 'RR_list_cor', 'nn20', 'nn50', 'poincare', 'breathing_signal', 'breathing_psd', 'breathing_frq', 'segment_indices'])

In [18]:
t_wd_dict = {'segment_indices': m_seg['segment_indices']}

df_wd = pd.DataFrame.from_dict(t_wd_dict)
df_m = pd.DataFrame(m_seg)

# merge the two dataframes with common column as segment_indices
df_segmented = pd.merge(df_m, df_wd, on='segment_indices')

# make the segment_indices as index with first value of it
df_segmented['segment_indices'] = df_segmented['segment_indices'].apply(lambda x: x[0])
df_segmented = df_segmented.set_index('segment_indices')

# remove index name
df_segmented.index.name = None


In [19]:
tt = df_segmented


# add new row with index 0
tt.loc[0] = np.nan


In [20]:
tt = tt.reindex(list(range(tt.index.min(),tt.index.max()+1)),fill_value='NaN')

tt

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9181496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# merge two dataframes with common index
pre_final = pd.merge(data.reset_index(), tt, left_index=True, right_index=True)
pre_final.tail()

,level_0,index,unixTimes,accelerometerX,accelerometerY,accelerometerZ,gyroscopeX,gyroscopeY,gyroscopeZ,ledGreen,...,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
9181496,18362992,1310204.5,1.648124e+12,-2192.50,159.75,3402.00,14.25,-199.25,61.50,-38.178333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181497,18362994,1310206.5,1.648124e+12,-2194.00,163.75,3395.50,19.75,-201.00,57.75,-29.258338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181498,18362996,1310208.5,1.648124e+12,-2193.75,167.00,3394.75,17.00,-222.25,58.00,-18.186857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181499,18362998,1310210.5,1.648124e+12,-2192.00,161.50,3398.00,22.75,-246.75,54.00,-7.657366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181500,18363000,1310212.5,1.648124e+12,-2187.25,163.00,3402.00,25.00,-237.25,58.50,0.566843,...,134.366369,222.843576,0.888889,0.777778,80.0,157.351188,119.835278,59238.572803,1.313062,0.20436


In [22]:
# select first 15000 rows
sed_df = pre_final
sed_df.at[1500, 'bpm']

70.45454545454545

In [23]:
sed_df

,level_0,index,unixTimes,accelerometerX,accelerometerY,accelerometerZ,gyroscopeX,gyroscopeY,gyroscopeZ,ledGreen,...,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,0,12734.5,1.644992e+12,57.00,296.25,4021.00,-201.75,-47.75,99.00,43.987752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,12736.5,1.644992e+12,29.00,302.50,4026.00,-226.25,-75.25,119.75,243.038014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,12738.5,1.644992e+12,40.50,310.25,4031.00,-228.00,-108.25,113.25,627.523616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,12740.5,1.644992e+12,53.50,310.00,4028.00,-224.50,-86.00,101.25,1035.918959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,12742.5,1.644992e+12,45.25,307.25,4026.75,-232.00,-86.25,116.50,1249.684019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9181496,18362992,1310204.5,1.648124e+12,-2192.50,159.75,3402.00,14.25,-199.25,61.50,-38.178333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181497,18362994,1310206.5,1.648124e+12,-2194.00,163.75,3395.50,19.75,-201.00,57.75,-29.258338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181498,18362996,1310208.5,1.648124e+12,-2193.75,167.00,3394.75,17.00,-222.25,58.00,-18.186857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9181499,18362998,1310210.5,1.648124e+12,-2192.00,161.50,3398.00,22.75,-246.75,54.00,-7.657366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
sed_df.to_csv('final_processed.csv')